![SPOTIFY](image.png) 

# Predicting Hits on Spotify using Machine Learning

#### Authors: Adithya Bhonsley, Prayaag Gupta, Benjamin Wang, Dinesh Polisetty 

In this era of digital music, understanding the dynamics of song popularity is more intriguing than ever! With millions of songs available at our fingertips, we were curious to leverage machine learning to launch a data-driven exploration into what makes songs hits. For our experiments, we'll take a closer look into Spotify, one of the largest music streaming platforms in the world, and aim to predict song popularity through a variety of different modeling and analysis techniques covered in class. Below is the methodology we follow throughout this notebook for our experiments:

* **Fetch**: Collect input data from various sources and synthesize
* **Modify**: Run data pre-processing, cleaning, and feature selection/engineering
* **Analyze**: Examine data using varying models, optimize through hyperparamater tuning, and compare
* **Discuss**: Present results and visualizations to draw meaningful insights

## Defining a Library

First, lets import all relevant libraries needed for data analysis, modeling, and visualizations.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Fetching Data

For our experiments, we'll use a Kaggle dataset sourced by developer Nidula Elgiryewithana, a comprehensive list of 2023 songs with an extensive insights on musical attributes, popularity, relevant metrics, and more. First, we read and load the data from its original CSV format.

In [2]:
data = pd.read_csv('data/spotify-2023.csv', encoding='latin-1')
data.head(10)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
5,Sprinter,"Dave, Central Cee",2,2023,6,1,2186,91,183706234,67,...,141,C#,Major,92,66,58,19,0,8,24
6,Ella Baila Sola,"Eslabon Armado, Peso Pluma",2,2023,3,16,3090,50,725980112,34,...,148,F,Minor,67,83,76,48,0,8,3
7,Columbia,Quevedo,1,2023,7,7,714,43,58149378,25,...,100,F,Major,67,26,71,37,0,11,4
8,fukumean,Gunna,1,2023,5,15,1096,83,95217315,60,...,130,C#,Minor,85,22,62,12,0,28,9
9,La Bebe - Remix,"Peso Pluma, Yng Lvcas",2,2023,3,17,2953,44,553634067,49,...,170,D,Minor,81,56,48,21,0,8,33


<hr style="border:0.25px solid gray">

## Data Exploration

Given the data, now let's take a closer look the features provided, their data-types, and other attributes which may need to adjusted or cleaned prior to experimentation.

In [ ]:
data.info()

Above we see various attributes regarding a song's factual attributes (name, artist, release time, charts ranking, etc) as well as some more subjective attributes like musical ratings (danceability, energy, etc). Here's a more in-depth breakdown on the meaning behind these subjective musical features, scored between 0 and 100:

* **danceability_%**: Measure of how suitiable a song is for dancing.
* **valence_%**: Measure of how upbeat a song is (low for sad/negative, high for positive/upbeat)	
* **energy_%**: Measure of how energetic a song is.
* **acousticness_%**: Measure of acoustic sound in the song	
* **instrumentalness_%**: Measure of instrumental content in the song	
* **liveness_%**: Measure of live performance elements in the song
* **speechiness_%**: Measure of spoken words in the song.

## Data Cleaning

#### Handling Missing Values
Let's inspect the dataset for any missing values which could potentially skew results ...

In [ ]:
data.isna().sum()

From the table above, we find 50 occurrences of missing <code>in_shazam_charts</code> values and 95 occurences of missing <code>key</code> values:

* The missing Shazam charts records are likely popular songs that did not chart on Shazam. We can mark these values with a placeholder of -1 since it's are outside the range of possible values (positive integers).
* The entries with missing key values, denoting the song's key, could likely not be established. These key values are assigned using pitch class labels (0-12), more info: <href>https://en.wikipedia.org/wiki/Pitch_class</href>. To address these missing values, we can fill with -1 as well since that is also outside the range of possible values.

In both cases, we resolve missing values using a placeholder value, clearly indicating to our future model that the data is missing or not applicable.

In [ ]:
# Fill missing values with -1 in shazam charts column
data['in_shazam_charts'] = data['in_shazam_charts'].fillna(-1)

# Fill missing values with -1 in key column
data['key'] = data['key'].fillna(-1)

# Check for missing again
data.isna().sum()

#### Handling Improper Types
Now let's check to see if any of our attributes are incorrectly typed in the data and require modifcation.

In [ ]:
data.dtypes

* Above, we see that <code>streams</code> is an object type, but instead should be numeric to represent a quantity. Additionally, the same applies with columns <code>in_deezer_playlists</code> and <code>in_shazam_charts</code>.
* The <code>key</code> and <code>mode</code> columns are categorical, denoting a 0-12 pitch label or Major/Minor label, respectively. We can convert these to integer representations.

In [ ]:
# Convert streams to numeric values
data['streams'] = pd.to_numeric(data['streams'], errors='coerce')

# Convert deezer playlist rankings to numeric values
data['in_deezer_playlists'] = pd.to_numeric(data['in_deezer_playlists'], errors='coerce')

# Convert shazam chart rankings to numeric values
data['in_shazam_charts'] = pd.to_numeric(data['in_shazam_charts'], errors='coerce')

# Assign integer representations for each pitch class
pitch_class_label = {'C': 0,
               'C#': 1,
               'D': 2,
               'D#': 3,
               'E': 4,
               'F': 5,
               'F#': 6,
               'G': 7,
               'G#': 8,
               'A': 9,
               'A#': 10,
               'B': 11
               }

# Assign integer representations for each mode class
mode_class_label = {"Major": 0, "Minor": 1}

# Convert key str values (A, B, C#, etc) -> (0, 1, ... 12)
data['key'] = data['key'].map(pitch_class_label).fillna(-1)

# Convert mode str labels to integer
data['mode'] = data['mode'].map(mode_class_label).fillna(-1)

## Feature Engineering

The first thing we want to do is create some features that are useful in predicting whether a song becomes extremely popular or not. 
* <code>Seasons</code> - This can be relevant as some seasons are associated with moods, themes, activities, cultures, etc.
* <code>Genre</code> - This can be relevant as some genres might resonate more with the average person
* <code>Artist Popularity</code> - This can be relevant as a song might gain a lot of traction based on an artist's popularity and the marketing they use for the song


In [ ]:
#Creating a seasons column and removing the released_month, released_day, and released_year columns
def month_to_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'

data['season'] = data['released_month'].apply(month_to_season)
columns_to_drop = ['released_year', 'released_month', 'released_day']
data = data.drop(columns=columns_to_drop)

data.head()

In [ ]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Getting the genre of a song and popularity of an artist with the spotify API
def read_spotify_credentials(file_path):
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            credentials[key] = value
    return credentials

credentials_file_path = 'spotify_credentials.txt'
spotify_credentials = read_spotify_credentials(credentials_file_path)
client_credentials_manager = SpotifyClientCredentials(
    client_id=spotify_credentials['client_id'],
    client_secret=spotify_credentials['client_secret']
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=5)

def get_top_genre(artist_name):
    results = sp.search(q='artist:' + artist_name, type='artist')
    #print(json.dumps(results, indent=4, sort_keys=True))

    if len(results['artists']['items']) > 0:
        artist_id = results['artists']['items'][0]['id']
        if len(sp.artist(artist_id)['genres']) > 0:
            return sp.artist(artist_id)['genres'][0]
        else:
            return None
    else:
        return None

def get_num_followers(artist_name):
    results = sp.search(q='artist:' + artist_name, type='artist')
    #print(json.dumps(results, indent=4, sort_keys=True))

    if len(results['artists']['items']) > 0:
        artist_id = results['artists']['items'][0]['id']
        num_followers = results['artists']['items'][0]['followers']['total']
        return num_followers
    else:
        return None

Since we do not want to call this API every time, we created a CSV that has all of our data cleaning changes with the number of followers that the artist has. Furthermore, although we also used the Spotify API to get the genre for each song, one-hot encoding this would significantly increase the dimensionality of the dataset. Thus, we decided to remove it. The cell below will fetch the data from our new CSV.

In [7]:
data = pd.read_csv('data_with_followers.csv', encoding='latin-1')

In [8]:
#data['genre'] = data['artist(s)_name'].apply(get_top_genre)
#data['num_followers'] = data['artist(s)_name'].apply(get_num_followers)
data.head()
data.isna().sum()

# Preserve copy of original
original = data.copy()

columns_to_drop = ['track_name', 'artist(s)_name', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'in_spotify_playlists', 'artist_count', 'in_spotify_charts']
data = data.drop(columns=columns_to_drop)
data.isna().sum()

streams               1
bpm                   0
key                   0
mode                  0
danceability_%        0
valence_%             0
energy_%              0
acousticness_%        0
instrumentalness_%    0
liveness_%            0
speechiness_%         0
season                0
num_followers         0
dtype: int64

As we can see, there is one record that has a null value from number of streams. We can just impute this with the median value.

In [9]:
#Filling null values

median_num_streams = data['streams'].median()
data['streams'].fillna(median_num_streams, inplace=True)

# There were also some records that had NAs for num_follwers. This is the code we wrote to impute those values as well.
median_followers = data['num_followers'].median()
data['num_followers'].fillna(median_followers, inplace=True)
data.head()
data.isna().sum()

streams               0
bpm                   0
key                   0
mode                  0
danceability_%        0
valence_%             0
energy_%              0
acousticness_%        0
instrumentalness_%    0
liveness_%            0
speechiness_%         0
season                0
num_followers         0
dtype: int64

Now, we have our dataset! This is stored in the processed_data variable.

In [10]:
processed_data = data.copy(deep=False)
processed_data.head()

,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,season,num_followers
0,141381703.0,125,11.0,0,80,89,83,31,0,8,4,Summer,12484102.0
1,133716286.0,92,1.0,0,71,61,74,7,0,10,4,Spring,13213551.0
2,140003974.0,138,5.0,0,51,32,53,17,0,31,6,Summer,29916000.0
3,800840817.0,170,9.0,0,55,58,72,11,0,11,15,Summer,92699744.0
4,303236322.0,144,9.0,1,65,23,80,14,63,11,6,Spring,76681922.0


In [12]:
import sklearn

columns_to_encode = ['season']
data_to_encode = processed_data[columns_to_encode]
encoder = sklearn.preprocessing.OneHotEncoder(sparse=False, drop='first')
encoded_data = encoder.fit_transform(data_to_encode)

# NOTE: get_feature_names() renamed to get_feature_names_out(); use latter for upgraded sklearn versions
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

processed_data = pd.concat([processed_data, encoded_df], axis=1)
processed_data.drop(columns=columns_to_encode, inplace=True)

processed_data.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,num_followers,season_Spring,season_Summer,season_Winter
0,141381703.0,125,11.0,0,80,89,83,31,0,8,4,12484102.0,0.0,1.0,0.0
1,133716286.0,92,1.0,0,71,61,74,7,0,10,4,13213551.0,1.0,0.0,0.0
2,140003974.0,138,5.0,0,51,32,53,17,0,31,6,29916000.0,0.0,1.0,0.0
3,800840817.0,170,9.0,0,55,58,72,11,0,11,15,92699744.0,0.0,1.0,0.0
4,303236322.0,144,9.0,1,65,23,80,14,63,11,6,76681922.0,1.0,0.0,0.0


In [13]:
from sklearn.model_selection import train_test_split

features = processed_data.drop(['in_spotify_charts'], axis=1)  # Exclude 'streams' and 'in_spotify_charts' columns
charts_label = processed_data['in_spotify_charts']

X_train, X_test, charts_train, charts_test = train_test_split(features, charts_label, test_size=0.2, random_state=1)
X_train.head()


KeyError: "['in_spotify_charts'] not found in axis"

#### Descriptive Statistics

In [ ]:
data.describe()

In [ ]:
# Notes about descriptive stats

#### Data Visualizations

Now, let's visualize these individual attributes to determine potentially important trends, outliers, and characteristics. First, let's take a look at the **top 20** songs on Spotify in 2023:

In [ ]:
# Sort and fetch top 10 songs
data_sorted = original.sort_values(by='streams', ascending=False)
top_20 = data_sorted.head(20)

norm = plt.Normalize(data['streams'].min(), data['streams'].max())
colors = plt.cm.rainbow(norm(top_20['streams'])) 

# Display results in bar chart
plt.subplots(1,1, figsize=(6,4))
sns.barplot(data=top_20, x='streams', y='track_name', palette=list(colors), hue='track_name')
plt.xlabel('Streams (in billions)')
plt.ylabel('Song Name')

plt.show()

Now, let's inspect (a) distribution of attribute values and (b) the relationships between all the attributes in our dataset to our prediction variable (# of streams) through several visualizations:

In [ ]:
plt.figure(figsize=(30,16))

for i, col in enumerate(data.columns):
    plt.subplot(4, 4, i + 1)
    sns.histplot(data=data, x=col, bins=20)
    plt.tight_layout
    
plt.show()

In [ ]:
plt.figure(figsize=(25,16))

exclude_cols = ["track_name", "artist(s)_name", "streams"]

for i, col in enumerate(data.columns):
    if col not in exclude_cols:
        plt.subplot(4,4, i)
        sns.scatterplot(data=data, x=col, y='streams')
    
plt.tight_layout()
plt.show()

In [ ]:
numeric_data = data.select_dtypes(include='number')
correlation_with_streams = numeric_data.corr()['streams'].drop('streams').sort_values(ascending=True)

heatmap_data = pd.DataFrame(correlation_with_streams).T

cmap = sns.diverging_palette(10, 133, as_cmap=True)

plt.figure(figsize=(20, 2))
sns.heatmap(heatmap_data, annot=True, cmap=cmap, center=0,
            cbar=False, linewidths=1, linecolor='black')

plt.xticks(rotation=45)
plt.xlabel('feature')
plt.ylabel('correlation')
plt.title('Identifying Feature Correlations to # of Streams')
plt.show()

<hr style="border:0.25px solid gray">

## Predictive Modeling

In [ ]:
# Simple Regression

In [ ]:
# Polynomial Regression ?

# Regression Trees

Now, let's look at another type of model called regression trees. First things first, we exprimented with a couple of columns to drop to see which columns weren't really relevant but it seems like all of the columns helped imrpoves the R2 value for the model. We also made sure to do 5-folds cross validation to find the best hyperparameters for the max_depth, min_samples_split, min_samples_leaf, as well as the criterion the model was evaluated on. From this, we were able to determine that there is indeed some connection between the features we had as well as engineered and with the popularity of the song, in this case the number of streams the song got. By printing out the regression tree and analyzing it, we realized that the num_followers lead to the least MSE and that 2 other features that also gave a good indication was the mode of the song as well as the liveness of the song.

In [23]:
# Regression Trees
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_text


model = DecisionTreeRegressor()
# X = processed_data.drop(columns=['streams', 'num_followers']).copy() 
# X = processed_data.drop(columns=['streams', 'mode']).copy() 
# X = processed_data.drop(columns=['streams', 'season_Spring', 'season_Fall', 'season_Summer', 'season_Winter']).copy() 
X = processed_data.drop(columns=['streams']).copy() 
y = processed_data['streams']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up the parameter grid
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['friedman_mse', 'poisson'],
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2')

# Fit the model
grid_search.fit(X, y)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# tree_rules = export_text(best_model)
# print(tree_rules)
# print(f"Mean Squared Error: {mse}")
# print(f"R-squared: {r2}")


Mean Squared Error: 2.6320188577666554e+17
R-squared: 0.028517448004328205


In [ ]:
# KNN Regression + hyperparamater tuning

#### Support Vector Regression
After scaling data and employing a 5-folds cross validation, we began the cross validation loop to find optimal hyperparameters. Across all iterations, Radial Basis Function was consistently the best kernel function. In regards to the C, epsilon, and gamma hyperparameters, there did not appear to be convergence at a global minimum. Rather, the best MSE values were in the range of 428 to 432. 

An MSE of 428, equating to an RSME of 20.68, demonstrates relatively excellent prediction for position on the Spotify charts. The downside of performing the SVR is that we are not given information about individual attribute influence.

In [ ]:
# SVR
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

param_grid = {
    'svm__C': [1, 5, 10],
    'svm__epsilon': [3, 5, 7],
     'svm__gamma': [0.1, 1, 5],
     'svm__kernel': ['poly', 'rbf', 'sigmoid']
}

svm_pipeline = Pipeline([('scaler', StandardScaler()), ('svm', SVR())])

# Set up GridSearchCV
grid_search = GridSearchCV(svm_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, charts_train)

print("Best Parameters:", grid_search.best_params_)

# Get the best model
best_model = grid_search.best_estimator_

# Predict on the test set
charts_pred = best_model.predict(X_test)

# Calculate mean squared error (MSE)
mse = mean_squared_error(charts_test, charts_pred)

print("Mean Squared Error on Test Set:", mse)

In [ ]:
# Evaluate + Ensemble

<hr style="border:0.25px solid gray">


## Results & Discussion